<a href="https://colab.research.google.com/github/dishaabhat/prm-sys/blob/main/llm_category.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [10]:
import os
import pandas as pd
import re
from groq import Groq

# Load API key
def load_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

api_key = load_api_key("/content/groq.txt")
os.environ["GROQ_API_KEY"] = api_key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Define category mappings
category_mappings = {
    'ach': ['income', 'other debit', 'bounce inward', 'loan'],
    'atm': ['withdrawal', 'charges'],
    'bank account': ['payin wallet'],
    'bill online': ['expenses'],
    'card': ['charges'],
    'cash': ['deposit'],
    'cheque': ['other credit', 'deposit', 'charges', 'bounce outward', 'bounce inward'],
    'cms': ['loan', 'loan disbursed', 'income'],
    'eba': ['charges'],
    'ecs': ['charges'],
    'imps': ['loan', 'other debit', 'Salary (analysed)', 'other credit', 'validation credit', 'reversal credit', 'bounce inward'],
    'inb': ['expenses', 'charges', 'other debit'],
    'inft': ['other credit', 'other debit'],
    'ips': ['expenses', 'reversal credit'],
    'mandate': ['charges'],
    'neft': ['loan disbursed', 'investment payout', 'other debit', 'income', 'bounce inward'],
    'net banking': ['investment payin'],
    'other': ['subsidy', 'loan', 'tax debit', 'income', 'withdrawal', 'loan disbursed', 'investment payout', 'donation', 'charges', 'expenses', 'validation credit'],
    'pcd': ['expenses'],
    'pci': ['expenses'],
    'pos': ['charges', 'expenses'],
    'rtgs': ['charges'],
    'sak': ['deposit'],
    'standing instruction': ['investment payin', 'bounce inward'],
    'terminal': ['card settlement'],
    'transfer': ['reimbursement', 'subsidy', 'loan', 'investment payout'],
    'upi': ['other credit', 'income', 'other debit', 'cashback', 'expenses', 'payout', 'to account debit', 'salary paid', 'reward', 'reversal', 'loan disbursed', 'loan', 'investment payin', 'insurance debit'],
    'visa': ['reversal credit', 'reversal'],
    'vps': ['other debit', 'reversal credit', 'expenses', 'reversal']
}

def determine_mode(description):
    description = description.lower()
    patterns = {
        'upi': r'upi|@upi', 'ips': r'ips', 'atm': r'atm|cash wdl|nfs|atmdec|atm/spcnd|atmfee',
        'other': r'apbs|ltgurxx', 'bill online': r'bill|rchg|recharge|vodafone|airtel',
        'vps': r'vps', 'inft': r'inft', 'ecs': r'ecs|ecs mandate|ecsrt|rt|rtnchg',
        'visa': r'visa', 'imps': r'imps|mmt/imps', 'cms': r'cms', 'cash': r'cash|by cash',
        'ach': r'ach', 'cheque': r'clg|chq|cheque|chrg|clg dep', 'card': r'dcardfee|gst for atm card',
        'pos': r'pos', 'transfer': r'transfer', 'bank account': r'bank account',
        'neft': r'neft', 'standing instruction': r'standing instruction|si',
        'net banking': r'net banking', 'pcd': r'pcd', 'pci': r'pci', 'inb': r'inb|ib',
        'sak': r'sak', 'rtgs': r'rtgs', 'eba': r'eba', 'mandate': r'mandate', 'terminal': r'terminal'
    }

    for mode, pattern in patterns.items():
        if re.search(pattern, description):
            return mode
    return 'other'

def predict_category(transaction, mode):
    # Use direct mapping if only one category is mapped to the mode
    if len(category_mappings.get(mode, [])) == 1:
        return category_mappings[mode][0]

    # Use LLM for ambiguous categories
    prompt = f"Classify the transaction based on mode, description, and amount into one of the following categories.Try to map the most appropriate category according to the available options from the mode mapping. Respond with only the category name, nothing else:\n\n"  \
             f"Mode: {transaction['mode']}\n" \
             f"Amount: {transaction['amount']}\n" \
             f"Description: {transaction['description']}\n" \
             f"Categories: {category_mappings.get(mode, [])}\n" \
             f"Respond with only the category name."

    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3-8b-8192"
    )
    return chat_completion.choices[0].message.content.strip()

# Load and classify transactions
file_path = '/content/trans2.xlsx'
df = pd.read_excel(file_path)
df['mode'] = df['Description'].apply(determine_mode)
df['category'] = df.apply(lambda row: predict_category(
    {'mode': row['mode'], 'amount': row['Amount'], 'description': row['Description']}, row['mode']), axis=1)

# Save the results
output_file_path = 'classified_transactions4.xlsx'
df.to_excel(output_file_path, index=False)

print(f'Classified transactions saved to {output_file_path}.')

Classified transactions saved to classified_transactions4.xlsx.


In [6]:
import os
import pandas as pd
import re
from groq import Groq

def load_api_key(file_path):
    with open(file_path, "r") as file:
        return file.read().strip()

api_key = load_api_key("/content/groq.txt")
os.environ["GROQ_API_KEY"] = api_key

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def determine_mode(description):
    description = description.lower()

    if re.search(r'upi|@upi', description):
        return 'upi'
    elif re.search(r'ips', description):
        return 'ips'
    elif re.search(r'atm|cash wdl|nfs|atmdec|atm/spcnd|atmfee', description):
        return 'atm'
    elif re.search(r'apbs|ltgurxx', description):
        return 'other'
    elif re.search(r'bill|rchg|recharge|vodafone|airtel', description):
        return 'bill online'
    elif re.search(r'vps', description):
        return 'vps'
    elif re.search(r'inft', description):
        return 'inft'
    elif re.search(r'ecs|ecs mandate|ecsrt|rt|rtnchg', description):
        return 'ecs'
    elif re.search(r'visa', description):
        return 'visa'
    elif re.search(r'imps|mmt/imps', description):
        return 'imps'
    elif re.search(r'cms', description):
        return 'cms'
    elif re.search(r'cash|by cash', description):
        return 'cash'
    elif re.search(r'ach', description):
        return 'ach'
    elif re.search(r'clg|chq|cheque|chrg|clg dep', description):
        return 'cheque'
    elif re.search(r'dcardfee|gst for atm card', description):
        return 'card'
    elif re.search(r'pos', description):
        return 'pos'
    elif re.search(r'transfer', description):
        return 'transfer'
    elif re.search(r'bank account', description):
        return 'bank account'
    elif re.search(r'neft', description):
        return 'neft'
    elif re.search(r'standing instruction|si', description):
        return 'standing instruction'
    elif re.search(r'net banking', description):
        return 'net banking'
    elif re.search(r'pcd', description):
        return 'pcd'
    elif re.search(r'pci', description):
        return 'pci'
    elif re.search(r'inb|ib', description):
        return 'inb'
    elif re.search(r'sak', description):
        return 'sak'
    elif re.search(r'rtgs', description):
        return 'rtgs'
    elif re.search(r'eba', description):
        return 'eba'
    elif re.search(r'mandate', description):
        return 'mandate'
    elif re.search(r'terminal', description):
        return 'terminal'
    else:
        return 'other'

possible_categories = [
    "income", "other debit", "bounce inward", "loan", "withdrawal", "charges", "payin wallet",
    "expenses", "deposit", "other credit", "bounce outward", "loan disbursed", "Salary (analysed)",
    "validation credit", "reversal credit", "investment payout", "investment payin", "subsidy",
    "tax debit", "donation", "card settlement", "reimbursement", "cashback", "payout",
    "to account debit", "salary paid", "reward", "reversal", "insurance debit"
]

def predict_category(mode, amount, description):
    prompt = (
    f"Please classify the following transaction details into one of the categories:\n\n"
    f"Mode: {mode}\nAmount: {amount}\nDescription: {description}\nCategory: \n\n"
    f"Possible Categories:\n"
    f"- income\n"
    f"- other debit\n"
    f"- bounce inward\n"
    f"- loan\n"
    f"- withdrawal\n"
    f"- charges\n"
    f"- payin wallet\n"
    f"- expenses\n"
    f"- deposit\n"
    f"- other credit\n"
    f"- bounce outward\n"
    f"- loan disbursed\n"
    f"- Salary (analysed)\n"
    f"- validation credit\n"
    f"- reversal credit\n"
    f"- investment payout\n"
    f"- investment payin\n"
    f"- subsidy\n"
    f"- tax debit\n"
    f"- donation\n"
    f"- card settlement\n"
    f"- reimbursement\n"
    f"- cashback\n"
    f"- payout\n"
    f"- to account debit\n"
    f"- salary paid\n"
    f"- reward\n"
    f"- reversal\n"
    f"- insurance debit"
)

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",  # replace with a suitable model name
    )

    predicted_category = chat_completion.choices[0].message.content.strip()

    for category in possible_categories:
        if category.lower() in predicted_category.lower():
            return category
    return "Unknown Category"

# Load the Excel file
file_path = '/content/test2.xlsx'
df = pd.read_excel(file_path)

df['mode'] = df['Description'].apply(determine_mode)
df['category'] = df.apply(lambda row: predict_category(row['mode'], row['Amount'], row['Description']), axis=1)

output_file_path = 'classified_transactions.xlsx'
df.to_excel(output_file_path, index=False)

print(f'Classified transactions saved to {output_file_path}.')

Classified transactions saved to classified_transactions.xlsx.
